In [18]:
import matplotlib.pyplot as plt
import numpy as np
import pompy.models
import pompy.processors
import nengo
%matplotlib inline
import matplotlib.pyplot as plt

In [27]:

class World(object):
    def __init__(self, x, y, heading,
                 bounds_x=(-0.5, 0.5),
                 bounds_y=(-0.2, 0.2),
                 grid_size=(100, 40),
                 source_location=(-0.5, 0),
                 acceleration=0.2,
                 drag=0.5,
                 puff_release_rate=100,
                 puff_init_rad=0.1,
                 centre_rel_diff_scale=1.5,
                 sensor_dist=0.03,
                 trail_length=0,
                 trail_dt=0.1,
                 plume_dt=0.005,
                 precompute_time=0.0,
                 max_concentration=1e5):

        self.heading = heading
        self.acceleration = acceleration
        self.velocity = [0, 0]
        self.drag = drag
        self.conc = None
        self.trail_length = trail_length
        self.trail_dt = trail_dt
        self.plume_dt = plume_dt
        self.last_trail_time = None
        self.trail = []

        self.x = x
        self.y = y

        self.max_concentration = max_concentration

        self.sensor_angle = 45 * np.pi / 180
        self.sensor_dist = sensor_dist

        # Define simulation region
        wind_region = pompy.models.Rectangle(-1, 10., -2., 2.)  # (0., -2., 10., 2.)
        sim_region = pompy.models.Rectangle(bounds_x[0], bounds_x[1], bounds_y[0], bounds_y[1])
        self.bounds_x = bounds_x
        self.bounds_y = bounds_y

        # Set up wind model
        wind_grid_dim_x = 21
        wind_grid_dim_y = 11
        wind_vel_x_av = 2
        wind_vel_y_av = 0.
        self.wind_model = pompy.models.WindModel(
            wind_region, wind_grid_dim_x, wind_grid_dim_y,
            wind_vel_x_av, wind_vel_y_av)

        # Set up plume model
        centre_rel_diff_scale = 1.5
        self.plume_model = pompy.models.PlumeModel(
            sim_region,
            (source_location[0], source_location[1], 0),
            self.wind_model,
            centre_rel_diff_scale=centre_rel_diff_scale,
            puff_release_rate=puff_release_rate,
            puff_init_rad=puff_init_rad)

        for i in range(int(precompute_time / plume_dt)):
            self.wind_model.update(plume_dt)
            self.plume_model.update(plume_dt)

        # Create a concentration array generator
        array_z = 0.01
        self.grid_size = grid_size
        puff_mol_amount = 1.
        self.array_gen = pompy.processors.ConcentrationArrayGenerator(
            sim_region, array_z, self.grid_size[0], self.grid_size[1], puff_mol_amount)
    def make_sensor(self):
        self.last_update = -9999
        update_rate = self.plume_dt
        dt = self.plume_dt

        def update(t):
            # print(t)
            # print(self.last_update + update_rate)
            if t >= self.last_update + update_rate:
                self.last_update = t
                print(dt)
                self.wind_model.update(dt)
                self.plume_model.update(dt)
                self.conc = self.array_gen.generate_single_array(self.plume_model.puff_array)

            lx, ly, rx, ry = self.sensor_pos()
            lx, ly = self.grid_location(lx, ly)
            rx, ry = self.grid_location(rx, ry)
            lx = int(lx)
            rx = int(rx)
            ly = int(ly)
            ry = int(ry)
            if 0 <= lx < self.grid_size[0] and 0 <= ly < self.grid_size[1]:
                lc = self.conc[lx, ly]
            else:
                lc = 0
            if 0 <= rx < self.grid_size[0] and 0 <= ry < self.grid_size[1]:
                rc = self.conc[rx, ry]
            else:
                rc = 0

            return (np.clip(lc / self.max_concentration, 0, 1),
                    np.clip(rc / self.max_concentration, 0, 1))

        return nengo.Node(update)

In [29]:
class World(object):
    
    def __init__(self, x, y, heading):
        self.x=x
        self.y=y
        self.heading=heading
        
        wind_region = pompy.models.Rectangle(x_min=-1., x_max=10, y_min=-2., y_max=2.)
        sim_region = pompy.models.Rectangle(x_min=-1, x_max=1, y_min=-1, y_max=1)
        # set up wind model
        wind_vel_x_av =2
        wind_vel_y_av =0.
        self.wind_model = pompy.models.WindModel(wind_region, 21, 11, wind_vel_x_av,wind_vel_y_av)

        centre_rel_diff_scale = 2
        puff_release_rate=10
        puff_init_rad=0.001

        self.plume_model = pompy.models.PlumeModel(
            sim_region, (-0.5, 0., 0.), wind_model
        , centre_rel_diff_scale ,
        puff_release_rate,
        puff_init_rad,)
    
        array_z = 0.01
        grid_size=[100,100]
        grid_size = grid_size
        puff_mol_amount = 1.
        self.array_gen = pompy.processors.ConcentrationArrayGenerator(
            sim_region, array_z, grid_size[0], 
        grid_size[1], puff_mol_amount)
        self.plume_dt = 0.005
        self.max_concentration=100
    def make_sensor(self):
        self.last_update = -9999
        update_rate = self.plume_dt
        dt = self.plume_dt

        def update(t):
            # print(t)
            # print(self.last_update + update_rate)
            if t >= self.last_update + update_rate:
                self.last_update = t
                print(dt)
                self.wind_model.update(dt)
                self.plume_model.update(dt)
                self.conc = self.array_gen.generate_single_array(self.plume_model.puff_array)

            lx, ly, rx, ry = self.sensor_pos()
            lx, ly = self.grid_location(lx, ly)
            rx, ry = self.grid_location(rx, ry)
            lx = int(lx)
            rx = int(rx)
            ly = int(ly)
            ry = int(ry)
            if 0 <= lx < self.grid_size[0] and 0 <= ly < self.grid_size[1]:
                lc = self.conc[lx, ly]
            else:
                lc = 0
            if 0 <= rx < self.grid_size[0] and 0 <= ry < self.grid_size[1]:
                rc = self.conc[rx, ry]
            else:
                rc = 0

            return (np.clip(lc / self.max_concentration, 0, 1),
                    np.clip(rc / self.max_concentration, 0, 1))

        return nengo.Node(update)
    def sensor_pos(self):
        left_x = self.x + self.sensor_dist * np.cos(self.heading - self.sensor_angle)
        left_y = self.y + self.sensor_dist * np.sin(self.heading - self.sensor_angle)
        right_x = self.x + self.sensor_dist * np.cos(self.heading + self.sensor_angle)
        right_y = self.y + self.sensor_dist * np.sin(self.heading + self.sensor_angle)
        return left_x, left_y, right_x, right_y



    def grid_location(self, x, y):
        tx = (x - self.bounds_x[0]) / (self.bounds_x[1] - self.bounds_x[0])
        ty = (y - self.bounds_y[0]) / (self.bounds_y[1] - self.bounds_y[0])
        return tx * self.grid_size[0], ty * self.grid_size[1]

In [30]:
model = nengo.Network()
with model:
    
    world = World(x=0.5, y=0, heading=np.pi)
    world.make_sensor()

0.005


AttributeError: 'WindModel' object has no attribute '_interp_u'